# Two-port Sparameter Graph

## Models that fit into this 
1. S2P on disk 
2. SNP / S2PV1 Classes in Touchstone Models
3. Jim Booth Format on Disk -> JBSparameter class
4. TwoPortNRRawModel and TwoPortRawModel
5. S2P on disk in different formats (AsciiDataTable)
6. XMLDataTables
7. XsltResultString an XslResultFile
8. CITI
9. VNATools

## Issues
1. When doing analysis the data must present a uniform format, i.e. units same, values are complex matrix or list
2. Header or Metadata is removed for processing
3. What is the result of a measurement or the result of a model or simulation

## Common Analysis
1. form a correction given a series of measured standards (requires more than one measurement, definition and some extra metadata)
2. Apply a correction to a uncorrected DUT measurement
3. Extract device parameters given the corrected DUT measurement, this assumes DUT fits a model
4. Track or form Uncertainty estimates in 1-3



In [1]:
from pyMeasure import *
from pyMeasure.Code.DataHandlers.GraphModels import *

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [2]:
def S2pFile_to_S2p(s2p_file_name):
    """Returns a S2p class given a S2p file name"""
    return S2PV1(s2p_file_name)

def S2p_to_S2pFile(s2p,s2p_file_name="test.s2p"):
    """Saves the s2p class as file_name"""
    s2p.save(s2p_file_name)
    return s2p_file_name

In [3]:
graph_options={"graph_name":"Two Port Graph",
                  "node_names":["S2pFile",'S2pv1'],
                  "node_descriptions":["S2P file on disk","S2p class in Touchstone.Models"],
                  "current_node":'S2pFile',
                  "state":[1,0],
                  "data":"test.s2p",
                  "edge_2_to_1":S2p_to_S2pFile,
                  "edge_1_to_2":S2pFile_to_S2p}
two_port_graph=Graph(**graph_options)

In [4]:
two_port_graph.show()

In [5]:
two_port_graph.move_to_node("S2pv1")
s2p=two_port_graph.data

['edge_S2pFile_S2pv1_000']
moving S2pFile -> S2pv1


In [6]:
two_port_graph.data.options["data_delimiter"]="  "

In [7]:
s2p.elements

['data', 'noiseparameter_data', 'comments', 'option_line']

In [8]:
two_port_graph.data.show()

In [43]:
def Model_to_File(model,file_path=None):
    """Uses the save method of a model to create file, default path is save default for the model"""
    if file_path:
        old_path=model.path
        model.path=file_path
        out=model.save()
        model.path=old_path
    else:
        out=model.save()
    return out

def File_to_Model(file_path,model_class_name):
    """Uses a class to open the File and returns an object of type model_class_name"""
    return globals()[model_class_name](file_path)
def Snp_to_AsciiDataTable(snp_model):
    """Transforms an SNP model to an AsciiDataTable"""
    # This translation can work for any subclass also...
    # First Parse the comments into inline and header, might have to drop the inline comments
    header=[]
    inline_comments=[]
    if snp_model.comments:
        for comment in snp_model.comments[:]:
            if comment[2]==0:
                header.append(comment[0])
            else:
                inline_comments.append(comment)
    column_units=[snp_model.frequency_units]+[None for i in range(len(snp_model.column_names)-1)]
    column_types=["float" for i in range(len(snp_model.column_names))]
    new_table=AsciiDataTable(None,header=header,data=snp_model.data,
                             column_names=snp_model.column_names,
                            column_units=column_units,column_types=column_types)
    return new_table

def Snp_to_AsciiDataTableSubClass(snp_model,model="JBSparameter"):
    """Transforms an SNP model to an AsciiDataTable"""
    # This translation can work for any subclass also...
    # First Parse the comments into inline and header, might have to drop the inline comments
    if model in ["JBSparameter"]:
        snp_model.change_data_format("RI")
        snp_model.change_frequency_units("Hz")
    header=[]
    inline_comments=[]
    if snp_model.comments:
        for comment in snp_model.comments[:]:
            if comment[2]==0:
                header.append(comment[0])
            else:
                inline_comments.append(comment)
    column_units=[snp_model.frequency_units]+[None for i in range(len(snp_model.column_names)-1)]
    column_types=["float" for i in range(len(snp_model.column_names))]
    model_class=globals()[model]
    new_table=model_class(None,header=header,data=snp_model.data,
                             column_names=snp_model.column_names,
                            column_units=column_units,column_types=column_types)

    return new_table    


In [ ]:
s2p.change_data_format

In [33]:
table=Snp_to_AsciiDataTableSubClass(s2p)

In [11]:
table.elements

['header', 'column_names', 'data', 'footer', 'inline_comments', 'metadata']

In [28]:
table.show()

In [13]:
xml=AsciiDataTable_to_XmlDataTable(table,style_sheet=os.path.join(TESTS_DIRECTORY,"../XSL/S2P_RI_STYLE.xsl"))

In [14]:
xml.show()

In [17]:
html=xml.to_HTML()

In [18]:
from pyMeasure.Code.DataHandlers.HTMLModels import *

The module lxml.html.cssselect was not found or had an error. If it is not installedInstall it with pip install cssselect


In [19]:
html_2=HTMLBase(None,html_content=html)

In [20]:
html_2.to_pdf()

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        


'HTML_Document_20161221_001.pdf'

In [21]:
html_2.show()


file://c:/users/sandersa/appdata/local/temp/1/tmpowb0it.html


In [55]:
jb_two_port=JBSparameter(os.path.join(TESTS_DIRECTORY,"ftest6_L1_g5_HF_air"))

Warning \n is in the remove tokens


In [56]:
jb_two_port.show()

In [57]:
s2p_2=JBSparameter_to_S2PV1(jb_two_port)

In [58]:
s2p_2.show()

In [59]:
jb_two_port_2=Snp_to_AsciiDataTableSubClass(s2p_2)

In [47]:
jb_two_port_2==jb_two_port

False

In [48]:
jb_two_port_2.show()

In [49]:
jb_two_port.show()

In [60]:
jb_two_port_2.header

['9/30/2015 5:10 PM DMM reading = 2.809244E+1; IFBW= 3.000000E+2 (Hz); Power= -2.000000E+1 (dBm);',
 'Q26Ra_g5 and the Song wafer with the pdms roof and su8 microfluidics. NDO, 2015/II/pg111, -20db']

In [61]:
jb_two_port.header

['9/30/2015 5:10 PM DMM reading = 2.809244E+1; IFBW= 3.000000E+2 (Hz); Power= -2.000000E+1 (dBm);',
 'Q26Ra_g5 and the Song wafer with the pdms roof and su8 microfluidics. NDO, 2015/II/pg111, -20db']

In [62]:
print jb_two_port.header

['9/30/2015 5:10 PM DMM reading = 2.809244E+1; IFBW= 3.000000E+2 (Hz); Power= -2.000000E+1 (dBm);', 'Q26Ra_g5 and the Song wafer with the pdms roof and su8 microfluidics. NDO, 2015/II/pg111, -20db']


In [65]:
from pyMeasure.Code.Utils.HelpUtils import *

In [66]:
import pdoc


In [68]:
doc=pdoc.html('pyMeasure.Code.DataHandlers.GeneralModels')

In [69]:
html_3=HTMLBase(None,html_content=doc)

In [70]:
html_3.to_pdf()

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        


'HTML_Document_20161221_001.pdf'

In [71]:
create_examples_page(r"Image_Graph_Example_20161201_001.ipynb")

In [72]:
html_4=HTMLBase(r"Image_Graph_Example_20161201_001.html")

In [73]:
html_4.to_pdf()

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        


'Image_Graph_Example_20161201_001.pdf'

In [74]:
create_help_page('pyMeasure.Code.DataHandlers.XMLModels')